<a href="https://colab.research.google.com/github/Rebex3000/variableextract/blob/main/AIRCentre_WorldclimLongtermTemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!DOCTYPE html>
<html>
<body>
  <h1>Extract longterm climate variables (1970-2000) from worldclim, with loop for 12 months </h1>
  <p>Fick, S.E. and R.J. Hijmans, 2017. WorldClim 2: new 1km spatial resolution climate surfaces for global land areas. International Journal of Climatology 37 (12): 4302-4315. </p>

  <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS5th4HWNhEfBMtu7blfu7FNLGvYbaljkWFRA&usqp=CAU">

  <p>Variable: average temperature (°C) between 1970 and 2000</p>
  <p>Resolution of tiff file: 30 seconds, approx. 1 km</p>
  <p>Source link: <a> https://www.worldclim.org/data/worldclim21.html</a></p>

  <p>Extract variables for whole countries based on country borders from shape file.</p>
  <p>Requirements:</p>
  <ul>
    <li>Shapefile with country borders of the whole world with column "AREAID"</li>
    <li>Table with column "AREAID"</li>
  </ul>
</body>
</html>


In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 106.6 MB/s eta 0:00:00


In [ ]:
pip install rasterstats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.3 MB/s eta 0:00:00


In [ ]:
pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import packages
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
import rasterio
import matplotlib.pyplot as plt
import numpy as np


# Test files
Own test file paths must be inserted here:

In [ ]:
# load test files with less entries
# table
# df = pd.read_csv('/content/drive/MyDrive/AIRCentre/introductions_test.csv')
# print(df.head())
# shapefile that has only some regions in Europe
#shapefile = gpd.read_file('/content/drive/MyDrive/AIRCentre/my_few_worldregions.shp')
# print(shapefile.head())

# Actual files
Own actual file paths must be inserted here:

In [ ]:
# load actual files
# table
df = pd.read_csv('/content/drive/MyDrive/AIRCentre/aedesalbopictus.csv')
print(df.head())
# shapefile with worldregions
shapefile = gpd.read_file('/content/drive/MyDrive/AIRCentre/my_global_regions_rebecca230303.shp')
print(shapefile.head())


   SPECIESID           SPECIES AREAID  ISO                            NAME_0  \
0          1  Aedes albopictus   A505  USA               USA (North America)   
1          2  Aedes albopictus   A501  BRA  Brazil (Central & South America)   
2          3  Aedes albopictus   A224  ALB                  Albania (Europe)   
3          4  Aedes albopictus   A328  ITA                    Italy (Europe)   
4          5  Aedes albopictus   A295  FRA                   France (Europe)   

   YEAR  PERIOD                               ORIGIN ESTABLISHED  \
0  1946    1940                   Philippines (Asia)           Y   
1  1986    1980                               (Asia)           Y   
2  1989    1980                         China (Asia)           Y   
3  1990    1990                  USA (North America)           Y   
4  1999    1990  Japan (Asia) or USA (North America)           Y   

  dispersal_pathway  
0        Used tyres  
1        Used tyres  
2        Used tyres  
3        Used tyres  


# Long term temperature from Worldclim
Variable: avergae temperature between 1970 and 2000 <br>
Resolution of tiff file: 30 sec., approx. 1km <br>
Source link:  https://www.worldclim.org/data/worldclim21.html <br>
Download 12 files (1 per month) from link, add to drive/local folder and paste file path here: <br>

In [ ]:
# # initialize the list of the respective annual average maximum temperatures
tavg = []

# # loop through each entry in the csv table
for index, row in df.iterrows():
    # extract relevant data from the row
    species_id = row['SPECIESID']
    species = row['SPECIES']
    area_id = row['AREAID']
    iso = row['ISO']
    country = row['NAME_0']
    year = row['YEAR']

    # select the relevant region from the shapefile
    region = shapefile.loc[shapefile['AREAID'] == area_id].iloc[0]

    # initialize list of monthly average temperatures
    monthly_tavg = []
    if pd.isnull(country):
        continue

    try:
        # loop through the 12 tiff files (1 each month) for the year
        for m in range(1, 13):
            print(m)
            # select the relevant tiff file for the month and year
            tiff_path = '/content/drive/MyDrive/AIRCentre/wc2.1_30s_tavg_{:02d}.tif'.format(m)


            # zonal statistics
            # extract the mean temperature for the region from the tiff file
            with rasterio.open(tiff_path) as src:
                affine = src.transform
                stats = zonal_stats(region.geometry, tiff_path, affine=affine, stats=['mean'])
                monthly_tavg.append(stats[0]['mean'])
                print(stats[0]['mean'])

        # calculate the yearly average temperature and append to the list of yearly values
        yearly_tavg = sum(monthly_tavg)/len(monthly_tavg)
        tavg.append(yearly_tavg)
        # print the result
        print(f"SpeciesID {species_id}, Country {country}, AreaID {area_id}, Year {year}, Mean Tavg {yearly_tavg}")

    except:
        # if url is not found or introduction later than 2019, in order to not lose the data
        tavg.append(9999)
        dferror = pd.DataFrame(tavg, columns=['TMEAN_LONG'])
        dferror.to_excel('/Users/rebeccapabst/Desktop/aircentre/MYDATABASE/results/tmean_long_except_result.xlsx', index=False)

# append mean tmax to initial table
df['TAVG_LONG'] = tavg


1
-5.700052729035297
2
-3.398524457332935
3
0.3958792091067433
4
5.925689732708177
5
12.035401187640394
6
17.28498408617573
7
19.985440768608058
8
19.046745079296834
9
14.69439775447168
10
7.9652244490151505
11
0.8656062962521356
12
-3.9258545964214626
SpeciesID 1, Country USA (North America), AreaID A505, Year 1946, Mean Tavg 7.097911398373768
1
25.050300357923263
2
25.02429022329086
3
24.89209670976888
4
24.43256866722535
5
23.561541654592304
6
22.735612972956037
7
22.577753933631474
8
23.61680751247249
9
24.512658892082612
10
25.08628848458267
11
25.196931415010848
12
25.110484723647215
SpeciesID 2, Country Brazil (Central & South America), AreaID A501, Year 1986, Mean Tavg 24.316444628932
1
2.8507049832755205
2
3.6570028127534013
3
6.196821644968015
4
9.912546316109559
5
14.6870185377061
6
18.365304588251195
7
20.803205806829443
8
20.595140891071267
9
17.150900418956663
10
12.47850453869718
11
7.7309259280115326
12
5.025447816244707
SpeciesID 3, Country Albania (Europe), AreaID A22

# Export Result

In [ ]:
# save the DataFrame to an Excel file
df.to_excel('/content/drive/MyDrive/AIRCentre/tavg_longterm.xlsx', index=False)
